#1. 主要内容
Requests模块<br/>
网页爬虫与向网页提交数据<br/>

###Requests
完美代替Python的urllib2模块<br/>
更强的自动化和更好的体验<br/>
更完善的功能（高级使用模式）<br/>

###第三方库安装技巧
少用easy_install，只能安装不方便卸载<br/>
尽量使用pip形式安装<br/>
如果出现bug，使用全能的模块网站：http://www.lfd.uci.edu/~gohlke/pythonlibs/

###单线程爬虫的基本原理
requests获取网页代码<br/>
<li>直接获取源代码</li>
<li>修改http头获取源代码</li>

用正则表达式予以辅助

###向网页提交数据
Get与Post<br/>
前提：分析目标网站，再决定是否用requests进行表单提交，以及具体提交哪些内容？<br/>
<li>Get  从服务器上获取数据，通过构造url中的参数来实现功能</li>
<li>Post  向服务器传送数据，将数据放在header里进行提交</li>

#2. 爬虫练习

目标网站：http://www.jikexueyuan.com/course/?pageNum=1<br/>
目标内容：课程名称、课程介绍、课程时间、课程等级、课程人数

In [10]:
import requests, re

class spider(object):
    def __init__(self):
        print '开始抓取……'
    
    def  source(self, url):
        html = requests.get(url)
        return html.text
    
    def pages(self, url, total_page):
        now_page = int(re.search('pageNum=(\d+)', url, re.S).group(1))
        page_group = []
        for i in range(now_page, total_page + 1):
            link = re.sub('pageNum=\d+', 'pageNum=%s' %i, url, re.S)
            page_group.append(link)
        return page_group
    
    def get_class(self, source):
        every_class = re.findall('style="height: 88px;">(.*?)</div>', source, re.S)
        return every_class
    
    def getinfo(self, eachclass):
        info = {}
        info['title'] = re.findall('>(.*?)</a></h2>', eachclass, re.S)[0].split('>')[1]
        info['intro'] = re.search('display: none;">(.*?)</p>', eachclass, re.S).group(1)
        info['people'] = re.search('<em class="learn-number">(.*?)</em>', eachclass, re.S).group(1).strip(" ")
        detail_field = re.findall('<em>(.*?)</em>', eachclass, re.S)
        info['time'] = detail_field[0].strip(" ")
        info['level'] = detail_field[1].strip(" ")
        return info
    
    def saveinfo(self, classinfo):
        f = open('D:/Python/crawler/jikexueyuan.txt', 'a')
        for each in classinfo:
            f.writelines('title: ' + each['title'].encode('utf-8') + '\n')
            f.writelines('content: ' + each['intro'].strip('\n').strip(' 			').encode('utf-8') + '\n')
            f.writelines('learn_num: ' + each['people'].encode('utf-8') + '\n')
            f.writelines('classtime: ' + each['time'].strip('\n').strip(' 			').encode('utf-8') + '\n')
            f.writelines('classlevel: ' + each['level'].encode('utf-8') + '\n\n')
        f.close()
        
if __name__ == "__main__":
    classinfo = []
    url = "http://www.jikexueyuan.com/course/?pageNum=1"
    spider_new = spider()
    all_links = spider_new.pages(url, 2)
    for link in all_links:
        print '正在爬取……' + link
        html = spider_new.source(link)
        everyclass = spider_new.get_class(html)
        for each in everyclass:
            info = spider_new.getinfo(each)
            classinfo.append(info)
    spider_new.saveinfo(classinfo)

开始抓取……
正在爬取……http://www.jikexueyuan.com/course/?pageNum=1
正在爬取……http://www.jikexueyuan.com/course/?pageNum=2


爬取日语学习网站<br/>
修改http头绕过反爬虫机制

In [8]:
import requests
html = requests.get('http://jp.tingroom.com/yuedu/yd300p/')
print type(html)
html.encoding = "utf-8"
print html.text

<class 'requests.models.Response'>
<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html xmlns="http://www.w3.org/1999/xhtml"><head>
<meta http-equiv="Content-Type" content="text/html; charset=utf-8" />
<title>300篇精选中日文对照阅读_日语阅读_日语学习网</title>
<meta name="keywords" content="日语阅读,日语文章阅读,中日对照阅读,300篇日文阅读"/>
<meta name="description" content="日语阅读,日语文章阅读,中日对照阅读,300篇日文阅读-日语学习网"/>
<meta name="mobile-agent" content="format=html5;url=http://jp.tingroom.com/wap/index.php?moduleid=21&catid=566">
<meta name="mobile-agent" content="format=xhtml;url=http://jp.tingroom.com/wap/index.php?moduleid=21&catid=566">
<link href="http://xyz.tingroom.com/skin/jp/jp.css" rel="stylesheet" media="screen" type="text/css" />
<link href="http://xyz.tingroom.com/skin/jp/layout.css" rel="stylesheet" media="screen" type="text/css" />
<link href="http://xyz.tingroom.com/skin/jp/dedecms.css" rel="stylesheet" media="screen" type="text/css" />
<link 

In [9]:
import requests, re
hat = {"User-Agent" : "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/49.0.2623.110 Safari/537.36"}
html = requests.get('http://jp.tingroom.com/yuedu/yd300p/', headers = hat)
html.encoding = "utf-8"
field = re.search('ul class="e2">(.*?)</ul>', html.text, re.S).group(1)
content_field = re.findall('padding-left:5px;">(.*?)</li>', field, re.S)
for each in content_field:
    title = re.search('color: #039;">(.*?)</a>', each, re.S).group(1)
    time= re.search('font-size:14px;">(.*?)</span>',each,re.S).group(1)
    intro = re.search('color:#666666;">(.*?)</span>', each, re.S).group(1)
    print "title: %s, \n time: %s, \n intro: %s \n" %(title, time, intro)

title: 300篇精选中日文对照阅读 289 挪威的森林（中日对照）第二章, 
 time: 2013-01-19, 
 intro: 第二章 昔々、といってもせいぜい二十年ぐらい前のことなのだ... 

title: 300篇精选中日文对照阅读 288  挪威的森林（中日对照）第一章, 
 time: 2013-01-19, 
 intro: 挪威的森林（中日对照） 内容简介： 汉堡机场一曲忧郁的《挪... 

title: 300篇精选中日文对照阅读 287  藤野先生——名文选读, 
 time: 2013-01-19, 
 intro: 藤野先生名文选读中日文对照 東京も格別のことはなかつた。上... 

title: 300篇精选中日文对照阅读 286  夏目漱石 我是猫 第一章, 
 time: 2013-01-19, 
 intro: 夏目漱石 我是猫(中日对照) 吾輩は猫である 夏目漱石 一 吾輩... 

title: 300篇精选中日文对照阅读 285 それお皿の絵柄よ, 
 time: 2013-01-19, 
 intro: それお皿の絵柄よ 足のケガで中クラスの総合病院に入院しまし... 

title: 300篇精选中日文对照阅读 284 つもり, 
 time: 2012-12-29, 
 intro: あるけちん坊（ぼう）な男がおりました。 毎日毎日，ご飯どき... 

title: 300篇精选中日文对照阅读 283 遺言（ゆいごん）, 
 time: 2012-12-29, 
 intro: あるところに，たいへんへそまがりな息子（むすこ）がおりま... 

title: 300篇精选中日文对照阅读 282 やぶ医者, 
 time: 2012-12-29, 
 intro: 向こうから，お医者（いしゃ）がやってきました。そこへ店（... 

title: 300篇精选中日文对照阅读 281 表札, 
 time: 2012-12-29, 
 intro: お盆休み（ぼんやすみ）に帰ってきた者（もの）同士（どうし... 

title: 300篇精选中日文对照阅读 280 ととの目, 
 time: 2012-12-29, 
 intro: 昔（むかし），三太（さんた）という，ばかな息子がおりまし... 

title: 30